# Tagging (Sentiment Analysis)

## Intro and how to classify simple text

Let's begin with running a LLM to perform classification of a given piece of text. We can then use this perform classification and sentiment analysis for a bunch of different use cases. You will see some of these applications already in amazon review summaries for example.

In [28]:
from langchain_ollama.llms import OllamaLLM

llm = OllamaLLM(model="llama3.1:8b")

In [29]:
llm.invoke("Sentiment analysis: 'You are the best LLM'")

'A glowing review!\n\nThe sentiment of this statement is **POSITIVE**.\n\nHere\'s a breakdown:\n\n* The phrase "best" implies that the person has a very high opinion of me.\n* The use of all capital letters emphasizes the enthusiasm and confidence behind the statement.\n* The fact that it\'s a direct compliment suggests that the speaker is genuinely satisfied with my performance.\n* There are no negative words or phrases to balance out the positivity, making this sentiment even more pronounced.\n\nOverall, the sentiment analysis score for this statement would be around 9/10 in terms of positivity.'

In [30]:
llm.invoke("""Sentiment analysis in one word: 'Was so excited to get these especially after 
            reading all the reviews but when i opened them i noticed the rings were splintering. 
            I tried them once and used the velcro wrap around to secure the extra length of straps in place, and it tore off immediately'""")

'**Disappointed**'

In [31]:
llm.invoke("Sentiment analysis in one word and return nothing else: 'Was so excited to get these especially after reading all the reviews but when i opened them i noticed the rings were splintering. I tried them once and used the velcro wrap around to secure the extra length of straps in place, and it tore off immediately'")

'Frustration.'

In [32]:
llm.invoke("Sentiment analysis in one word positive, negative or neutral and return nothing else: 'Was so excited to get these especially after reading all the reviews but when i opened them i noticed the rings were splintering. I tried them once and used the velcro wrap around to secure the extra length of straps in place, and it tore off immediately'")

'Negative'

In [33]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain_ollama import ChatOllama

tagging_prompt = ChatPromptTemplate.from_template(
    """
Extract the desired information from the following passage.

Only extract the properties mentioned in the 'Classification' function.

Passage:
{input}
"""
)


class Classification(BaseModel):
    sentiment: str = Field(description="The sentiment of the text")
    aggressiveness: int = Field(
        description="How aggressive the text is on a scale from 1 to 10"
    )
    language: str = Field(description="The language the text is written in")


llm = ChatOllama(model="llama3.1:8b").with_structured_output(
    Classification
)

tagging_chain = tagging_prompt | llm

In [34]:
inp = """Was so excited to get these especially after reading all the reviews but when i opened them i noticed the rings were splintering. 
        I tried them once and used the velcro wrap around to secure the extra length of straps in place, and it tore off immediately"""
tagging_chain.invoke({"input": inp})

Classification(sentiment='negative', aggressiveness=7, language='english')